In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm          # это просто библиотека для отображения прогресс бара

Считываем CSV

In [2]:
df = pd.read_csv('recom.csv', sep=';', encoding='PT154')

In [3]:
df.head()

,order_number,order_date,client_id,item_name,item_quantity,level1,level3
0,198672,2019-05-01,14631,ПД-11-12,"10,90",одежные ткани,подкладочные
1,198672,2019-05-01,14631,ПД-17-3,"10,00",одежные ткани,подкладочные
2,198672,2019-05-01,14631,ПД-17-9,"10,00",одежные ткани,подкладочные
3,198672,2019-05-01,14631,ПД-3-1,"10,00",одежные ткани,подкладочные
4,198672,2019-05-01,14631,ПД-3-16,"10,00",одежные ткани,подкладочные


Т.к. количество через запятую - меняем ее на точку.

In [4]:
df['item_quantity'] = df['item_quantity'].replace(
    to_replace=r',', value='.', regex=True).astype('float32')

Удаляем ненужные столбцы

In [5]:
del df['order_number']
del df['order_date']
del df['level3']

Группируем по категриям товаров

In [6]:
c = df.groupby(['level1', 'client_id', 'item_name'], as_index=False).agg('sum')
c.head()

,level1,client_id,item_name,item_quantity
0,выкройки online,14614,ВКК-1-1,1.0
1,выкройки online,14614,ВКК-1-2,1.0
2,выкройки online,14614,ВКК-1-3,1.0
3,выкройки online,14614,ВКК-1-4,1.0
4,выкройки online,14614,ВКК-1-5,1.0


Отбираем только нужные нам категории

In [7]:
fabric = c.loc[(c.level1 == 'ткани для дома') | (c.level1 == 'одежные ткани') | (
    c.level1 == 'технические ткани') | (c.level1 == 'мебельные ткани'), ['client_id', 'item_name', 'item_quantity']]
fabric.head()

,client_id,item_name,item_quantity
1937,14795,ГЖ-7-1,29.400000
1938,14795,ГМ-50-1,31.299999
1939,14795,ГМ-55-1,34.000000
1940,14795,РМ-8-1,28.400000
1941,17255,ГМ-51-1,27.000000


Создаем функцию, которая присваивает ранг товару в зависимости от того насколько товар интересен клиенту

In [8]:
def scores(df, id_client):
    b = df.loc[df.client_id == id_client]
    b['score'] = np.ceil(b['item_quantity'] / b['item_quantity'].max() * 10)
    return b

Создаем пустую табличку с нужными нам колонками

In [9]:
res = pd.DataFrame(columns=('client_id', 'item_name', 'item_quantity', 'score'))

In [10]:
res.head()

,client_id,item_name,item_quantity,score


Получаем список всех клиентов

In [11]:
clients_list = fabric.client_id.unique()
clients_list.shape

(2539,)

И заполняем табличку по каждому клиенту оценками каждому товару, который он покупал

In [12]:
for i in tqdm(clients_list):
    res = res.append(scores(fabric, i), ignore_index=True)

  0%|          | 0/2539 [00:00<?, ?it/s]/home/ssv/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
100%|██████████| 2539/2539 [01:50<00:00, 22.98it/s]


In [13]:
res.shape, fabric.shape   # убедились что все записи попали в табличку (просто есть клиенты, которые брали другие товары)

((18107, 4), (18107, 3))

Удаляем ненужные колонки

In [14]:
del res['item_quantity']

Получаем такую табличку,в которой по каждому клиенту есть его оценка товара

In [17]:
res.head(50)

,client_id,item_name,score
0,14795,ГЖ-7-1,3.0
1,14795,ГМ-50-1,3.0
2,14795,ГМ-55-1,3.0
3,14795,РМ-8-1,3.0
4,14795,АКС-1-41,3.0
5,14795,АКС-1-42,3.0
6,14795,АО-8-28,3.0
7,14795,АО-8-70,3.0
8,14795,АО-9-56,3.0
9,14795,ДЖП-104-2,3.0


Сохранились

In [16]:
res.to_csv('res.csv')